In [ ]:
import os
import cv2
import numpy as np
from keras_facenet import FaceNet
import requests

# ESP32-CAM setup
stream_url = "http://192.168.203.115:81/stream"
control_url = "http://192.168.203.115/control?led="
relay_on = False

# Load FaceNet
embedder = FaceNet()

# Load Haar Cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Face cropper
def detect_face(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    if len(faces) == 0:
        return None, None
    x, y, w, h = faces[0]
    return frame[y:y+h, x:x+w], (x, y, w, h)

# Cosine similarity
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Face recognizer
def recognize_face(embedding, database, threshold=0.6):
    best_score = -1
    best_match = "Unknown"
    for name, embeddings in database.items():
        scores = [cosine_similarity(embedding, e) for e in embeddings]
        avg_score = np.mean(scores)
        if avg_score > best_score:
            best_score = avg_score
            best_match = name
    return (best_match if best_score >= threshold else "Unknown", best_score)

# Load known embeddings
def load_database(path='dataset'):
    db = {}
    for person in os.listdir(path):
        person_dir = os.path.join(path, person)
        embeddings = []
        for img_file in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            face, _ = detect_face(img)
            if face is not None:
                face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                emb = embedder.embeddings([face_rgb])[0]
                embeddings.append(emb)
        if embeddings:
            db[person] = embeddings
    return db

# Load DB
print(" Loading face database...")
face_db = load_database()

# Connect to stream
print(" Connecting to ESP32-CAM...")
stream = requests.get(stream_url, stream=True, timeout=5)
bytes_buffer = b''

print(" Connected! Starting recognition...")
try:
    for chunk in stream.iter_content(chunk_size=1024):
        bytes_buffer += chunk
        a = bytes_buffer.find(b'\xff\xd8')
        b = bytes_buffer.find(b'\xff\xd9')
        if a != -1 and b != -1:
            jpg = bytes_buffer[a:b+2]
            bytes_buffer = bytes_buffer[b+2:]
            frame = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            if frame is None:
                continue

            face, coords = detect_face(frame)
            if face is not None:
                face_rgb = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
                emb = embedder.embeddings([face_rgb])[0]
                name, score = recognize_face(emb, face_db)

                x, y, w, h = coords
                color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
                cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
                cv2.putText(frame, f"{name} ({score:.2f})", (x, y-10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

                if name != "Unknown" and not relay_on:
                    requests.get("http://192.168.203.115/control?led=on")
                    relay_on = True
                    print(f" {name} recognized → Relay ON")
                elif name == "Unknown" and relay_on:
                    requests.get("http://192.168.203.115/control?led=off")
                    relay_on = False
                    print(" Unknown → Relay OFF")
            else:
                if relay_on:
                    requests.get(control_url + "off")
                    relay_on = False
                    print(" No face → Relay OFF")

            cv2.imshow("ESP32-CAM + FaceNet", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

except KeyboardInterrupt:
    print("\n Interrupted by user.")

except Exception as e:
    print(" Error:", e)

finally:
    cv2.destroyAllWindows()
    try:
        stream.close()
    except:
        pass